## RQ1 Creating FAO forest map

[Add Description]

Need to create a FAO Definition Approximation following the steps from Johnson et al (2023)

Steps:
1. Data Preparation
    1. Clip the 5 GER LULC shapefiles to the Natura 2000 areas
    2. Merge the clipped 5 GER LULC shapefiles into 1 shapefile
    3. Vectorise the clipped 5m JAXA raster
2. Intersect JAXA and GER LULC shapefiles
3. Conditional Reclassing
4. Convert to 5m raster

This doc has a raster to vector section: https://py.geocompx.org/05-raster-vector